# MATH 8650 Advanced Data Structures
## Term Project: Optimization of Bellman Ford Algorithm
### Submitted by:
### Vivek Koodli Udupa (CUID: C12768888)
### Sai Harsha Nagabothu (CUID: C11086803)
### Date: Nov - 30, 2018


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
from collections import deque
from pylab import rcParams
import numpy as np
import random
import timeit
import networkx as nx
from collections import defaultdict

### Priority Queue
Below is the implementation of Priority queues used in Dijkstra's path finding algorithm

In [5]:
class PriorityQueue():
    '''
    The arguments passed to a PriorityQueue must consist of
    objects than can be compared using <.
    Use a tuple (priority, item) if necessary.
    '''

    def __init__(self):
        self._pq = []

    def _parent(self,n):
        return (n-1)//2

    def _leftchild(self,n):
        return 2*n + 1

    def _rightchild(self,n):
        return 2*n + 2

    def push(self, obj):
        # append at end and bubble up
        self._pq.append( obj )
        n = len(self._pq)
        self._bubble_up(n-1)

    def _bubble_up(self, index):
        while index>0:
            cur_item = self._pq[index]
            parent_idx = self._parent(index)
            parent_item = self._pq[parent_idx]
            
            if cur_item < parent_item:
                # swap with parent
                self._pq[parent_idx] = cur_item
                self._pq[index] = parent_item
                index = parent_idx
            else:
                break

    def pop(self):
        n = len(self._pq)
        if n==0:
            return None
        if n==1:
            return self._pq.pop()
        
        # replace with last item and sift down:
        obj = self._pq[0]
        self._pq[0] = self._pq.pop()
        self._sift_down(0)
        return obj

    def heapify(self, items):
        self._pq = items
        index = self._parent(len(self._pq)-1)
        
        while index >=0:
            self._sift_down(index)
            index -= 1
    
    def _sift_down(self,index):
        n = len(self._pq)
        
        while index<n:           
            cur_item = self._pq[index]
            lc = self._leftchild(index)
            if n <= lc:
                break

            # first set small child to left child:
            small_child_item = self._pq[lc]
            small_child_idx = lc
            
            # right exists and is smaller?
            rc = self._rightchild(index)
            if rc < n:
                r_item = self._pq[rc]
                if r_item < small_child_item:
                    # right child is smaller than left child:
                    small_child_item = r_item
                    small_child_idx = rc
            
            # done: we are smaller than both children:
            if cur_item <= small_child_item:
                break
            
            # swap with smallest child:
            self._pq[index] = small_child_item
            self._pq[small_child_idx] = cur_item
            
            # continue with smallest child:
            index = small_child_idx
        
    def size(self):
        return len(self._pq)
    
    def is_empty(self):
        return len(self._pq) == 0
    
    def show(self):
        k=1
        for i in range(0,len(self._pq)):
            if i==k:
                print()
                k=k*2+1
            print(self._pq[i], end="")
        print()
        
    def decrease_priority(self, old, new):
        assert(new <= old)
        
        for i in range(0,len(self._pq)):
            if self._pq[i] == old:
                self._pq[i] = new
                self._bubble_up(i)
                return
        assert(false)

### Graph class
Graph class defines the vertices, edges and neighborhood of every vertex in the network. 

In [6]:
class Graph(object):
    '''Represents a graph'''

    def __init__(self, vertices, edges):
        '''A Graph is defined by its set of vertices
           and its set of edges.'''
        self.V = set(vertices) # The set of vertices
        self.E = set([])       # The set of edges
        self.neighbors = {}          # A dictionary that will hold the list
                               # of adjacent vertices for each vertex.
        self.vertex_coordinates = {}       # An optional dictionary that can hold coordinates
                               # for the vertices.
        self.edge_labels = {}  # a dictionary of labels for edges

        self.add_edges(edges)  # Note the call to add_edges will also
                               # update the neighbors dictionary
        print ('(Initializing a graph with %d vertices and %d edges)' % (len(self.V),len(self.E)))


    def add_vertices(self,vertex_list):
        ''' This method will add the vertices in the vertex_list
            to the set of vertices for this graph. Since V is a
            set, duplicate vertices will not be added to V. '''
        for v in vertex_list:
            self.V.add(v)
        self.build_neighbors()


    def add_edges(self,edge_list):
        ''' This method will add a list of edges to the graph
            It will insure that the vertices of each edge are
            included in the set of vertices (and not duplicated).
            It will also insure that edges are added to the
            list of edges and not duplicated. '''
        for s,t in edge_list:
            if (s,t) not in self.E and (t,s) not in self.E:
                self.V.add(s)
                self.V.add(t)
                self.E.add((s,t))
        self.build_neighbors()


    def build_neighbors(self):
        self.neighbors = {}
        for v in self.V:
            self.neighbors[v] = []
        for e in self.E:
            s,t = e
            self.neighbors[s].append(t)
            #self.neighbors[t].append(s)

### Network class

Network class is used to assign the given weights to the edges in the network

In [7]:
class Network(Graph):    
    def __init__(self, vertices, edge_weights):
        ''' Initialize the network with a list of vertices
        and weights (a dictionary with keys (E1, E2) and values are the weights)'''

        edges = []
        for e1,e2 in edge_weights:
            edges.append((e1,e2))
        
        Graph.__init__(self, vertices, edges)
        self.weights = {}
        for e1,e2 in edge_weights:
            weight = edge_weights[(e1,e2)]
            self.weights[(e1,e2)] = weight
            #self.weights[(e2,e1)] = weight
        self.edge_labels = self.weights

### Bellman Ford Algorithm

* Bellman Ford algorithm calculates the shortest path from the source node to all other nodes in the graph. 
* The graph may contain negative weight edges. 
* Negative Cycles cause the algorithm to go into an infinite loop. Such cycles are detected and reported
* The output of the algorithm is the minimum distance from source to all nodes. 

In [8]:
def BellmanFord(network, source):
    
    #Number of vertices
    vertCount = len(network.V)
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #To check for end of updates
    update = True
    
    #Check every vertix and relax nodes V - 1 times
    for i in range(vertCount - 1):
        if not update:
            return distance, vertCount
        else:
            update = False
            for e in network.E:
                u,v = e
                if not np.isinf(distance[u]) and distance[v] > distance[u] + network.weights[(u,v)]:
                    distance[v] = distance[u] + network.weights[(u,v)]
                    update = True
    
    #Negative Cycle check
    #Any update after V - 1 iterations indicate negative cycles
    for e in network.E:
        u,v = e
        if not np.isinf(distance[u]) and distance[v] > distance[u] + network.weights[(u,v)]:
            print("\nNegative Cycle Detected\n")
            return distance, vertCount
        
#     printSol(distance, vertCount)
    return distance, vertCount
    
def printSol(distance, count):
    """Prints the shortest distance from source to all otehr vertices in the graph"""
    print("\nVertix distance from source")
    print("----------------------------")
    print("Vertix \t \t Distance")
    print("------ \t \t --------")
    for i in range(count):
        print("  %c \t\t   %d" % ( i+65, distance[i]))

### SPFA
* Shortest Path faster algorithm is an optimization of bellman ford algorithm. 
* Here instead of relaxing all the nodes , only the nodes that are updated are relaxed in the next iteration.
* every relaxed node is added into a queue. And only the entries in the queue are processed in the next iteration.
* The order in which the nodes are processed sometimes helps the algorithm in prevent unnecessary relaxing. 
* The nodes are processed in three different orders in this project: 
    1. Using a FIFO queue, first updated node is processed first
    2. Using a LIFO queue, last updated node is processed first
    3. Using PAPE's queue, nodes updated for the first time are processed first and then after the first iteration, LIFO ordered is followed.

In [9]:
def SPFA_FIFO(network, source):
    """SPFA with FIFO queues"""
    #vertices count
    vertCount = len(network.V)
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #Initialize the visited vertex array
    visited = [0] * vertCount
    
    #A doubly ended queue to store visited vertices
    Q = deque();
    
    #Initialize the deque by adding the source node
    Q.append(source)
    
    #Run as long as the queue is not empty
    while(not isEmpty(Q)):
        
        x = Q.popleft()
        visited[x] = visited[x] + 1 #Visited count
        if(visited[x] > vertCount):
            print("\nNegative Cycles Found\n")
            return distance, vertCount
        else:
            distance, Q = relax(network, x, distance, Q)
            
    return distance, vertCount

def relax(network, x, distance, Q):
    """Relaxes all neighbours of vertex x"""
    for i in network.neighbors[x]:
        if distance[i] > distance[x] + network.weights[(x,i)]:
            distance[i] = distance[x] + network.weights[(x,i)]
            if not(i in Q):
                Q.append(i)
    return distance, Q

def isEmpty(queue):
    if(queue):
        return 0 #Not Empty
    else:
        return 1 #Empty

### SPFA LIFO

In [10]:
def SPFA_LIFO(network, source):
    """SPFA with LIFO queues"""
    #vertices count
    vertCount = len(network.V)
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #Initialize the visited vertex array
    visited = [0] * vertCount
    
    #A doubly ended queue to store visited vertices
    Q = deque();
    
    #Initialize the deque by adding the source node
    Q.append(source)
    
    #Run as long as the queue is not empty
    while(not isEmpty(Q)):
        
        x = Q.pop()
        visited[x] = visited[x] + 1 #Visited count
        if(visited[x] > vertCount):
            print("\nNegative Cycles Found\n")
            return distance, vertCount
        else:
            distance, Q = relax(network, x, distance, Q)
            
    return distance, vertCount

def relax(network, x, distance, Q):
    """Relaxes all neighbours of vertex x"""
    for i in network.neighbors[x]:
        if distance[i] > distance[x] + network.weights[(x,i)]:
            distance[i] = distance[x] + network.weights[(x,i)]
            if not(i in Q):
                Q.append(i)
    return distance, Q

def isEmpty(queue):
    if(queue):
        return 0 #Not Empty
    else:
        return 1 #Empty

### SPFA PAPE

In [11]:
def SPFA_PAPE(network, source):
    """SPFA with FIFO queues"""
    #vertices count
    vertCount = len(network.V)
    
    #Initialize distance to infinity and source distance to 0
    distance = [np.inf] * vertCount
    distance[source] = 0
    
    #Initialize the visited vertex array
    visited = [0] * vertCount
    
    #Number of times the vertex has been added to the queue
    queueCount = [0] * vertCount
    
    #A doubly ended queue to store visited vertices
    Q = deque();
    
    #Initialize the deque by adding the source node
    Q.append(source)
    queueCount[source] = queueCount[source] + 1 
    
    #Run as long as the queue is not empty
    while(not isEmpty(Q)):
        
        x = Q.popleft()
        visited[x] = visited[x] + 1 #Visited count
        if(visited[x] > vertCount):
            print("\nNegative Cycles Found\n")
            return distance, vertCount
        else:
            distance, Q = relax_pape(network, x, distance, Q, queueCount)
            
    return distance, vertCount

def relax_pape(network, x, distance, Q, queueCount):
    """Relaxes all neighbours of vertex x"""
    for i in network.neighbors[x]:
        if distance[i] > distance[x] + network.weights[(x,i)]:
            distance[i] = distance[x] + network.weights[(x,i)]
            if not(i in Q):
                if(queueCount[i] == 0):
                    Q.append(i)
                else:
                    Q.appendleft(i)
                queueCount[i] = queueCount[i] + 1
                    
    return distance, Q

def isEmpty(queue):
    if(queue):
        return 0 #Not Empty
    else:
        return 1 #Empty

### Dijkstras
* Dijkstra's algorithm is another path finding algorithm that performs much better than the Bellman Ford algorithm.
* But the drawback of Dijkstra's is that the edge weights must only be positive

In [ ]:
def dijkstra(network, source):
    dist = {source:0}
    prev = {}
    done = {}
    pq = PriorityQueue()
    pq.push((0,source))
    
    while not pq.is_empty():
        dist_u, u = pq.pop()
        if u in done:
            continue
        done[u] = True
        
        for v in network.neighbors[u]:
            new_dist_to_v = dist_u + network.weights[(u,v)]
            if not v in dist or dist[v]>new_dist_to_v:
                dist[v] = new_dist_to_v
                prev[v] = u
                pq.push((new_dist_to_v, v))
                
                
    return dist, prev
            
# dist, prev = dijkstra(G1,0)           
# print ("distance:", dist)
# print ("prev:", prev)
    

## Generating Random Graphs
* Random graphs with random weights are generated to test the above implemented algorithms

In [ ]:
nodes = 5000 #Number of nodes
edges = (nodes * (nodes - 1) ) // 2 #number of edges
# edges = nodes - 1

#Generate random graph using networkx
g=nx.gnm_random_graph(nodes,edges)

#Add random weight to the edges
for (u,v,w) in g.edges(data=True):
    w[(u, v)] = random.randint(-10,50)

    #convert the edge and weight into a dictionary of the form { (edge1, edge2): weight }
W = {}
x = []

for (u,v,w) in g.edges(data=True):
    x = list(w.values())
    W[(u,v)] = x[0]
# print(W)

#Make a list of vertices
V = list(range(0, nodes))
# print(V)

#Generate the network
G3 = Network(V,W)

#Performance analysis
%timeit distance, count = dijkstra(G3,0)
%timeit distance, count = BellmanFord(G3,0)
%timeit distance, count = SPFA_FIFO(G3,0)
%timeit distance, count = SPFA_LIFO(G3,0)
%timeit distance, count = SPFA_PAPE(G3,0)

print("Done")

#Printing the shortest paths
# printSol(distance,count)

### Test case 1 : Normal graph without negative cycles

In [ ]:
V = [0, 1, 2, 3, 4]
# W = {(0, 2):3, (0, 1):6, (0, 3):7, (1, 2):1, (1, 3):2, (2, 4):10, (1, 4):4}
W = {(0, 1):-1, (0, 2):4, (1,2):3, (1,3):2, (1,4):2, (3,2):5, (3,1):1, (4,3):-3}
G1 = Network(V,W)
print (G1.weights)

distance, count = BellmanFord(G1,0)
printSol(distance,count)

### Test case 2: Negative Cycles

In [ ]:
V = [0, 1, 2, 3]
W = {(0, 1):5, (0, 2):4, (1,3):3, (2,1):-6, (3,2):2}
G2 = Network(V,W)
print (G2.weights)

distance, count = dijkstra(G2,0)
printSol(distance,count)